In [1]:
!pip install mediapipe

In [39]:
import mediapipe as mp
import cv2

import csv
import os
import numpy as np
import pickle
import pandas as pd
import math

In [4]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [7]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    
    #this depends on the output video that is going to be put | the video and the internal camera need to be with the same size
    cap.set(3, 1280)
    cap.set(4, 720)
    
    print(cap.get(3))
    print(cap.get(4))
    print(cap.get(cv2.CAP_PROP_FPS))
        
    while cap.isOpened():
        ret, frame = cap.read()
        

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #ROI get
        #image1 = image[307:478, 591:714]
        
        #Adjust the results and the mediapipe to only watch at a given ROI (not in the whole image/video)
        results = holistic.process(image)
        #print(results.pose_landmarks)
            
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
                    
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Hand Detection 
         # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()

640.0
480.0
1280.0
720.0
30.0


In [36]:
landmarks = results.left_hand_landmarks.landmark

AttributeError: 'NoneType' object has no attribute 'landmark'

In [37]:
landmarks[mp_holistic.HandLandmark.PINKY_TIP.value]

x: 0.5179898142814636
y: 0.8276535868644714
z: -0.05491213873028755

In [38]:
landmarks[mp_holistic.HandLandmark.WRIST.value]

x: 0.596295177936554
y: 0.9249984622001648
z: -7.186736183939502e-07

In [52]:
def calculate_distance(p1, p2):
    
    distance = math.sqrt( ((int(p1[0])-int(p2[0]))**2)+((int(p1[1])-int(p2[1]))**2) )
    return int(distance)

In [49]:
#JUST FOR TESTING

wrist = [landmarks[mp_holistic.HandLandmark.WRIST.value].x,landmarks[mp_holistic.HandLandmark.WRIST.value].y]

pinky = [landmarks[mp_holistic.HandLandmark.PINKY_TIP.value].x,landmarks[mp_holistic.HandLandmark.PINKY_TIP.value].y]

#Position of displaying the detector 
position_top = list(np.multiply(pinky, [1280, 720]).astype(int))

position_bottom = list(np.multiply(wrist, [1280, 720]).astype(int))


In [50]:
#JUST FOR TESTING 

distance = calculate_distance(position_top, position_bottom)

print(distance)

122


## Detection only on the hands

In [69]:
cap = cv2.VideoCapture(0)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    
    #this depends on the output video that is going to be put | the video and the internal camera need to be with the same size
    cap.set(3, 1280)
    cap.set(4, 720)
    
    print(cap.get(3))
    print(cap.get(4))
    print(cap.get(cv2.CAP_PROP_FPS))
        
    while cap.isOpened():
        ret, frame = cap.read()
        

        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        #ROI get
        #image1 = image[307:478, 591:714]
        
        #Adjust the results and the mediapipe to only watch at a given ROI (not in the whole image/video)
        results = holistic.process(image)
        #print(results.pose_landmarks)
            
        # Recolor image back to BGR for rendering
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #Extracting the coordinates
        try:
            landmarks = results.left_hand_landmarks.landmark

    #             wrist_left = [landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_holistic.PoseLandmark.LEFT_WRIST.value].y]
    #             wrist_right = [landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_holistic.PoseLandmark.RIGHT_WRIST.value].y]
            wrist = [landmarks[mp_holistic.HandLandmark.WRIST.value].x,landmarks[mp_holistic.HandLandmark.WRIST.value].y]

            middle_finger = [landmarks[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP.value].x,landmarks[mp_holistic.HandLandmark.MIDDLE_FINGER_TIP.value].y]

            #Position of displaying the detector 
            position_top = list(np.multiply(middle_finger, [1280, 720]).astype(int))

            position_bottom = list(np.multiply(wrist, [1280, 720]).astype(int))

            #Getting the distance between the finger and the wrist 
            distance = calculate_distance(position_top, position_bottom)

            position_top[0] = position_top[0] - int(distance/2)
            position_bottom[0] = position_bottom[0] + int(distance/2)

            #Drawing a Ractangle on the hand
            cv2.rectangle(image, position_top, position_bottom, (0,0,255), 2)

        except:
            pass
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Hand Detection 
         # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break


cap.release()
cv2.destroyAllWindows()

640.0
480.0
1280.0
720.0
30.0


In [63]:

cap.release()
cv2.destroyAllWindows()

In [19]:
cv2.rectangle??